# Demonstration of coincidence window influence
In LAFOV, the coincidence window is often smaller than the maximum length. In this notebook, we explore how this influences the sensitivity image. You can use it as a basis for projections.

This notebook uses sirf.STIR but also the stir module. The latter is a SWIG-generated Python interface to STIR C++ code.

Author: Kris Thielemans  
First version: 19 May 2024

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2024 University College London.

This is software developed for the Collaborative Computational Project in Synergistic Reconstruction for Biomedical Imaging (http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

# Initial set-up

make sure figures appear inline and animations work

In [ ]:
%matplotlib widget

If the above failed, or you experience problems plotting, you should restart the Python kernel and use the following line instead.

In [ ]:
## %matplotlib inline

In [ ]:
import notebook_setup

#%% Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil

### Use the 'pet' prefix for all STIR-based SIRF functions
This is done here to explicitly differentiate between SIRF pet functions and anything else.

In [ ]:
import sirf.STIR
from sirf.Utilities import examples_data_path

### redirect STIR messages to some files
STIR output can be a bit verbose. There are currently also some spurious warnings. To avoid distracting you, we redirect these messages to some files. You can check these if things go wrong.


In [ ]:
_ = sirf.STIR.MessageRedirector('info.txt', 'warnings.txt')

### import the STIR Python interface directly

In [ ]:
import stir
import stirextra

### Set up the data and working directory for the notebook.

Please make sure that you have run the `download_data.sh` script first. See the [Introductory/introduction notebook](../Introductory/introduction.ipynb) for more information.

In [ ]:
from sirf_exercises import exercises_data_path
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('PET', 'LAFOV')

OK. finally done with initial set-up...

## Construct some data for a TOF scanner

In [ ]:
print(stir.Scanner.list_all_names())

In [ ]:
scanner = stir.Scanner(stir.Scanner.PETMR_Signa)
print(scanner.parameter_info())

In [ ]:
span=11
max_ring_diff=10
view_mash_factor=2
templ = pet.AcquisitionData('GE Signa PET/MR', span=span, max_ring_diff=max_ring_diff, view_mash_factor=view_mash_factor, tof_mash_factor=scanner.get_max_num_timing_poss())
templ.write('example.hs')

In [ ]:
templ_image = sirf.STIR.ImageData(templ)

In [ ]:
print(templ_image.get_geometrical_info().get_info())

In [ ]:
help(templ_image.zoom_image)

In [ ]:
dims=templ_image.dimensions()
templ_image2=templ_image.zoom_image(zooms=(1,.3, .3), size=(dims[0], dims[1]//3, dims[2]//3))
dims2=templ_image2.dimensions()
print(templ_image2.get_geometrical_info().get_info())

## Create sirf.STIR acquisition model

In [ ]:
am = sirf.STIR.AcquisitionModelUsingRayTracingMatrix()

In [ ]:
am.set_up(templ, templ_image2)

The `AcquisitionModel` is now ready for use.

Compute the sensitivity image

In [ ]:
ones = templ.get_uniform_copy(1)

In [ ]:
sens_image=am.backward(ones)

In [ ]:
sens_arr = sens_image.as_array()

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(sens_arr[dims2[0]//2, :,:])
plt.subplot(1,2,2)
plt.imshow(sens_arr[:, dims2[1]//2,:])
plt.show()

## Change the number of TOF bins (essentially the same as the coincidence window)

We cannot do that from SIRF, so have to directly use STIR. An alternative is to edit the text files by hand.

STIR Python is a bit different (and less friendly) than sirf.STIR.

In [ ]:
scanner.set_max_num_timing_poss(100)

In [ ]:
help(stir.ProjDataInfo.construct_proj_data_info)

In [ ]:
proj_data_info = stir.ProjDataInfo.construct_proj_data_info(scanner, span, max_ring_diff, view_mash_factor,scanner.get_max_num_non_arccorrected_bins(), False, scanner.get_max_num_timing_poss())

In [ ]:
stir_templ = stir.ProjDataInMemory(stir.ExamInfo(stir.ImagingModality(stir.ImagingModality.PT)), proj_data_info)
stir_templ.write_to_file('example2.hs')

We can now read this back into a sirf.STIR object

In [ ]:
templ = pet.AcquisitionData('example2.hs')

Now you should be ready to copy some of the previous lines to compute a new sensitivity image.

Once you have done that, we recommend to incorporate attenuation into the acquisition model. Check the PET introductory notebooks on how to do that.